# EJERCICIO M4 - S3

In [58]:
import pandas as pd

# 1. Cargar los datos desde el archivo CSV y analizar el total de ventas por categoría de producto
df_ventas = pd.read_csv("ventas_detalladas.csv")  
#print(df_ventas)
# Agregamos la columna Total Venta = cantidad_vendida*precioU
df_ventas['Total Venta'] = df_ventas['Cantidad Vendida'] * df_ventas['Precio Unitario']
#print(df_ventas)

# Agrupar por categoría para calcular el total vendido por cada una
ventas_por_categoria = df_ventas.groupby('Categoría')['Total Venta'].sum().reset_index()

print(ventas_por_categoria)



     Categoría  Total Venta
0     Deportes     56910989
1  Electrónica     47621041
2        Hogar     49961243
3     Juguetes     64399747
4         Ropa     46272203


In [54]:
# 2. Leer el archivo Excel y calcular el margen de ganancia promedio de cada categoría.

df_finanzas = pd.read_excel("reporte_financiero.xlsx")
#print(df_finanzas)

margen_promedio = df_finanzas.groupby('Categoría')['Margen de Ganancia'].mean().reset_index() #se puede realizar sin el .reset_index()
print(margen_promedio)

     Categoría  Margen de Ganancia
0     Deportes                0.24
1  Electrónica                0.19
2        Hogar                0.13
3     Juguetes                0.28
4         Ropa                0.38


In [44]:
# 3. Cargar los precios de la competencia
df_competencia = pd.read_csv("precios_competencia.csv")
print(df_competencia)

      Categoría        Producto  Precio Competencia
0   Electrónica        Notebook              133775
1   Electrónica      Smartphone              260563
2   Electrónica          Tablet              234069
3          Ropa          Polera              213987
4          Ropa        Pantalón               70043
5          Ropa        Chaqueta              102667
6         Hogar         Lámpara              277009
7         Hogar            Mesa               16591
8         Hogar           Silla               66190
9      Juguetes          Muñeca              198479
10     Juguetes  Auto a control              120844
11     Juguetes          Puzzle              198004
12     Deportes           Balón              292180
13     Deportes           Pesas              167536
14     Deportes       Bicicleta              287664


In [78]:
# 4. Comparar los precios de la competencia con los datos de ventas y margen de ganancia para determinar si es necesario ajustar los precios de la empresa
# Calculamos el precio promedio por producto en nuestra empresa
df_precio_promedio_empresa = df_ventas.groupby(['Categoría', 'Producto'])['Precio Unitario'].mean().reset_index()
#print(precio_promedio_empresa)

# Unimos los precios internos con los de la competencia
comparacion_precios = pd.merge(df_precio_promedio_empresa, df_competencia, on=['Categoría', 'Producto'], how='inner')
#print(comparacion_precios)

# Calculamos la diferencia de precios y sugerimos ajustes
comparacion_precios['Diferencia de Precio'] = comparacion_precios['Precio Unitario'] - comparacion_precios['Precio Competencia']
#print(comparacion_precios)

comparacion_precios['Recomendacion'] = comparacion_precios['Diferencia de Precio'].apply(
    lambda x: 'Bajar precio' if x > 0 else 'Subir precio'
)
#print(comparacion_precios.head(10))

# Agregamos el margen de ganancia por categoría a la tabla de comparación
df_comparacion_final = pd.merge(comparacion_precios, margen_promedio, on='Categoría', how='left')
print(df_comparacion_final)

      Categoría        Producto  Precio Unitario  Precio Competencia  \
0      Deportes           Balón    142434.521739              292180   
1      Deportes       Bicicleta    180105.142857              287664   
2      Deportes           Pesas    144732.583333              167536   
3   Electrónica        Notebook    175992.750000              133775   
4   Electrónica      Smartphone    157802.809524              260563   
5   Electrónica          Tablet    158327.466667              234069   
6         Hogar         Lámpara    191934.041667              277009   
7         Hogar            Mesa    148756.842105               16591   
8         Hogar           Silla    137279.736842               66190   
9      Juguetes  Auto a control    161587.142857              120844   
10     Juguetes          Muñeca    181592.428571              198479   
11     Juguetes          Puzzle    204257.562500              198004   
12         Ropa        Chaqueta    133422.666667              10

In [80]:
# 5 exportar a excel
# opción 1

# Guardar cada DataFrame como un archivo Excel independiente

ventas_por_categoria.to_excel("ventas_por_categoria.xlsx", index=False)
margen_promedio.to_excel("margen_promedio.xlsx", index=False)
comparacion_final.to_excel("comparacion_precios.xlsx", index=False)

In [82]:
# 5 exportar a excel
# opción 2

with pd.ExcelWriter("informe_resultados_marketing.xlsx") as writer:
    ventas_por_categoria.to_excel(writer, sheet_name='Ventas por Categoría', index=False)
    margen_promedio.to_excel(writer, sheet_name='Margen Promedio', index=False)
    comparacion_final.to_excel(writer, sheet_name='Comparación Precios', index=False)



In [84]:
# 5 exportar a excel
# opción 3

def formatear_pesos(col):
    return col.apply(lambda x: f"$ {x:,.0f}".replace(",", ".") if pd.notnull(x) else x)

ventas_por_categoria['Total Venta'] = formatear_pesos(ventas_por_categoria['Total Venta'])
comparacion_final['Precio Unitario'] = formatear_pesos(comparacion_final['Precio Unitario'])
comparacion_final['Precio Competencia'] = formatear_pesos(comparacion_final['Precio Competencia'])
comparacion_final['Diferencia de Precio'] = formatear_pesos(comparacion_final['Diferencia de Precio'])

# ------------------------------
# 6. Exportar a archivo Excel
# ------------------------------
with pd.ExcelWriter("informe_resultados_marketing_formateado.xlsx") as writer:
    ventas_por_categoria.to_excel(writer, sheet_name='Ventas por Categoría', index=False)
    margen_promedio.to_excel(writer, sheet_name='Margen Promedio', index=False)
    comparacion_final.to_excel(writer, sheet_name='Comparación Precios', index=False)